# Краулер, собирающий тексты горскопов

In [1]:
import requests
from bs4 import BeautifulSoup
import re
from tqdm import tqdm

In [2]:
signs = [
    'taurus',
    'aries',
    'gemini',
    'cancer',
    'leo',
    'virgo',
    'libra',
    'scorpio',
    'sagittarius',
    'capricorn',
    'aquarius',
    'pisces'
]

In [3]:
horo_texts = []

Собираю тексты по разным сайтам

In [4]:
for sign in tqdm(signs):
    link = f'https://1001goroskop.ru/?znak={sign}&kn=week'
    html = requests.get(link).text
    soup = BeautifulSoup(html, 'html.parser')
    horo = soup.find("div", {"itemprop": "description"})
    soup = BeautifulSoup(str(horo), 'html.parser')
    texts = soup.find_all("p", {})
    texts = [text.text for text in texts]
    horo_texts.extend(texts)

100%|██████████| 12/12 [00:19<00:00,  1.61s/it]


In [5]:
for sign in tqdm(signs):
    link = f'https://1001goroskop.ru/?znak={sign}&kn=tomorrow'
    html = requests.get(link).text
    soup = BeautifulSoup(html, 'html.parser')
    horo = soup.find("div", {"itemprop": "description"})
    soup = BeautifulSoup(str(horo), 'html.parser')
    texts = soup.find_all("p", {})
    texts = [text.text for text in texts]
    horo_texts.extend(texts)

100%|██████████| 12/12 [00:21<00:00,  1.78s/it]


In [6]:
for sign in tqdm(signs):
    link = f'https://1001goroskop.ru/?znak={sign}'
    html = requests.get(link).text
    soup = BeautifulSoup(html, 'html.parser')
    horo = soup.find("div", {"itemprop": "description"})
    soup = BeautifulSoup(str(horo), 'html.parser')
    texts = soup.find_all("p", {})
    texts = [text.text for text in texts]
    horo_texts.extend(texts)

100%|██████████| 12/12 [00:20<00:00,  1.71s/it]


In [7]:
link = 'https://www.chita.ru/horoscope/month/'
html = requests.get(link).text
soup = BeautifulSoup(html, 'html.parser')
horo = soup.find_all("span", {})
texts = [h.text for h in horo if len(h.text.split()) > 5][:-1]
horo_texts.extend(texts)

In [8]:
for sign in tqdm(signs):
    for time in tqdm(['yesterday', 'today', 'tomorrow', 'week', 'month', 'year']):
        link = f'https://horo.mail.ru/prediction/{sign}/{time}/'
        html = requests.get(link).text
        soup = BeautifulSoup(html, 'html.parser')
        horo = soup.find("div", {"class": "article__item article__item_alignment_left article__item_html"})
        horo_texts.append(horo)

100%|██████████| 12/12 [02:24<00:00, 12.03s/it]


In [9]:
for time in tqdm(['week', 'month']):
    link = f'http://new-year.dvorec.ru/{time}.php'
    html = requests.get(link).text
    soup = BeautifulSoup(html, 'html.parser')
    horo = soup.find_all("table", {"border": "0", "width": "100%", "cellspacing": "0", "cellpadding": "0"})
    text = horo[5].text
    text = re.sub('\n\n.+?на неделю', '', text)
    text = re.sub('\n\n.+?на месяц', '', text)
    horo_texts.append(text)


for time in tqdm(['yesterday', 'today', 'tomorrow']):
    link = f'http://new-year.dvorec.ru/{time}.php'
    html = requests.get(link).text
    soup = BeautifulSoup(html, 'html.parser')
    horo = soup.find_all("table", {"border": "0", "width": "100%", "cellspacing": "0", "cellpadding": "0"})
    text = horo[6].text
    text = re.sub('\n\n.+?на сегодня', '', text)
    text = re.sub('\n\n.+?на вчера', '', text)
    text = re.sub('\n\n.+?на завтра', '', text)
    horo_texts.append(text)

100%|██████████| 3/3 [00:04<00:00,  1.49s/it]


In [10]:
link = 'https://www.newwoman.ru/goroskop.html'
html = requests.get(link)
html = html.content.decode('utf-8')
soup = BeautifulSoup(html, 'html.parser')
horo = soup.find_all("span", {"style": "color: #000000;"})
for h in horo:
    if len(h.text.split()) > 5:
        horo_texts.append(h.text)

Удаляю дубликаты.

In [12]:
horo_texts = list(set(horo_texts))

In [13]:
len(' '.join(horo_texts).split())

28268

Тестирую цепь.

In [14]:
import markovify
train = ' '.join(horo_texts)
m = markovify.Text(train)

In [15]:
for i in range(10):
    sentence = m.make_short_sentence(max_chars=100)
    while sentence in train:
        sentence = m.make_short_sentence(max_chars=100)
    print(sentence)

Вам должно хватить веры в свои сугубо личные дела.
Сегодня в течение года будет несколько проще преодолеть преграды.
Особое внимание стоит обратить на них внимание.
При всех своих секретов и не ждите помощи даже от тех, кого вы, в свое время, очень поддержали.
В нем появятся новые люди, с которыми не нужно ломать голову.
От этого будет зависеть от того, какая ситуация будет играть ему в голову!
Хорошо, если в них режиссерскую роль.
Планы Девы будут отличаться логичностью и неожиданностью ходов.
Именно в это время вероятнее депрессии.
Потому что в этот период вы рискуете потерять шанс реализовать представившиеся возможности.


Записываю данные в json.

In [16]:
import json

In [19]:
with open('list_for_models.json', 'w', encoding='utf-8') as f:
    json.dump(horo_texts, f, ensure_ascii=False)